In [ ]:
import requests
import json
import pandas as pd
import time
from datetime import datetime
from os import path
import requests
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np; np.random.seed(0)
from datetime import date
import scipy.stats

In [ ]:
from __future__ import print_function
import matplotlib.ticker as ticker
from colour import Color

In [ ]:
color1 = '#%02x%02x%02x' % (25, 114, 120)
color2 = '#%02x%02x%02x' % (237, 221, 212)
color3 = '#%02x%02x%02x' % (196, 69, 54)

## HOPR - ETH - BTC databases

#### The dataframes are based on the daily, aggregated data provided by Uniswap about the three token's price and about the daily trading volume, liquidity and transactions of HOPR. 

#### HOPR

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

data_h = '''
query {
tokenDayDatas(where:{token: "0xf5581dfefd8fb0e4aec526be659cfab1f8c781da"}, orderBy:date, orderDirection:desc)
  {
    id
    date
    priceUSD
    totalLiquidityToken
    totalLiquidityUSD
    totalLiquidityETH
    dailyVolumeETH
    dailyVolumeToken
    dailyVolumeUSD
    dailyTxns
  }
}
'''

response_h = requests.post(url,json={'query': data_h})
data_h = json.loads(response_h.text)

In [ ]:
df_h = pd.DataFrame((pd.DataFrame(data_h)).iloc[0][0])

In [ ]:
df_h2 = df_h.rename(columns = {'id': 'id_h', 'date': 'date', 'priceUSD': 'priceUSD_h',
    'totalLiquidityToken': 'totalLiquidityToken_h',
    'totalLiquidityUSD': 'totalLiquidityUSD_h',
    'totalLiquidityETH': 'totalLiquidityETH_h',
    'dailyVolumeETH': 'dailyVolumeETH_h',
    'dailyVolumeToken': 'dailyVolumeToken_h',
    'dailyVolumeUSD': 'dailyVolumeUSD_h',
    'dailyTxns': 'dailyTxns_h' }, inplace = False)

In [ ]:
#final HOPR dataframe = df_h2
df_h2.head(5)

#### ETH 

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

data_e = '''
query {
tokenDayDatas(where:{token: "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"}, orderBy:date, orderDirection:desc)
  {
    id
    date
    priceUSD

  }
}
'''
response_e = requests.post(url,json={'query': data_e})
data_e = json.loads(response_e.text)

In [ ]:
df_e = pd.DataFrame((pd.DataFrame(data_e)).iloc[0][0])

In [ ]:
df_e2 = df_e.rename(columns = {'id': 'id_e', 'date': 'date', 'priceUSD': 'priceUSD_e'}, inplace = False)

In [ ]:
#final ETH dataframe = df_e2
df_e2.head(5)

#### BTC (wrapped bitcoin)

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
#url = 'https://api.thegraph.com/subgraphs/name/ianlapham/uniswapbackup'

data_b = '''
query {
tokenDayDatas(where:{token: "0x2260fac5e5542a773aa44fbcfedf7c193bc2c599"}, orderBy:date, orderDirection:desc)
  {
    id
    date
    priceUSD

  }
}
'''
response_b = requests.post(url,json={'query': data_b})
data_b = json.loads(response_b.text)

In [ ]:
df_b = pd.DataFrame((pd.DataFrame(data_b)).iloc[0][0])

In [ ]:
df_b2 = df_b.rename(columns = {'id': 'id_b', 'date': 'date', 'priceUSD': 'priceUSD_b'}, inplace = False)

In [ ]:
#final BTC dataframe = df_b2
df_b2.head(5)

In [ ]:
df_mer = pd.merge(pd.merge(df_h2,df_e2,on='date'),df_b2,on='date')

In [ ]:
df_mer[['dailyVolumeETH_h', 'dailyVolumeToken_h', 'dailyVolumeUSD_h', 'priceUSD_h', 'totalLiquidityETH_h', 'totalLiquidityToken_h',
       'totalLiquidityUSD_h', 'priceUSD_e', 'priceUSD_b', 'date', 'dailyTxns_h']] = df_mer[['dailyVolumeETH_h', 'dailyVolumeToken_h', 'dailyVolumeUSD_h', 'priceUSD_h', 'totalLiquidityETH_h', 'totalLiquidityToken_h',
       'totalLiquidityUSD_h', 'priceUSD_e', 'priceUSD_b', 'date', 'dailyTxns_h']].apply(pd.to_numeric)

In [ ]:
df_mer['date_r'] = pd.to_datetime(df_mer['date'], unit='s').apply(lambda x: x.date())

In [ ]:
import calendar

In [ ]:
df_mer["weekday"] = df_mer['date_r'].apply(lambda x: calendar.day_name[x.weekday()])

In [ ]:
#merged dataframe, containing the data of the HOPR, ETH, BTC tokens - df_mer
df_mer.head(5)

## HOPR - ETH/BTC price 

In [ ]:
price_corr = df_mer[["priceUSD_h", "priceUSD_e", "priceUSD_b"]]

In [ ]:
plt.figure(figsize=(5,5))
heatmap = sns.heatmap(price_corr.corr(), vmin=-1, vmax=1, annot=True)
plt.show()

#### HOPR - ETH price

In [ ]:
d_e = df_mer['priceUSD_e']
d_h = df_mer['priceUSD_h']
d_b = df_mer['priceUSD_b']
time = df_mer['date_r']
weekday = df_mer['weekday']
usdvol = df_mer['dailyVolumeUSD_h']
Txns = df_mer['dailyTxns_h']

In [ ]:
t = list(time)

In [ ]:
fig, axs = plt.subplots(2, sharex = True, figsize=(12, 8))
axs[0].plot(time, d_h, color=color1)
axs[0].set_title('HOPR daily USD price')
axs[1].plot(time, d_e, color=color3)
axs[1].set_title('ETH daily USD price')

In [ ]:
plt.figure(figsize=(20,7))

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)
ax1.plot(time, d_h, color=color1)
ax1.set_xlabel('date')
ax1.set_ylabel('HOPR price in USD', color=color1)
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(time, d_e, color=color3)
ax2.set_ylabel("ETH price in USD", color=color3)

fig1.set_tight_layout(True)
plt.show()

In [ ]:
## Scaled price chart

In [ ]:
from sklearn.preprocessing import QuantileTransformer

In [ ]:
df_sc = df_mer[["priceUSD_h", 'priceUSD_e', 'priceUSD_b']]

In [ ]:
scaler = QuantileTransformer(output_distribution='uniform')
scaled_df = scaler.fit_transform(df_sc)
scaled_df = (pd.DataFrame(scaled_df, columns=df_sc.columns)).reset_index()

In [ ]:
ind_ti = (df_mer['date_r']).reset_index()

In [ ]:
df_scd = pd.merge(scaled_df,ind_ti,on='index')

In [ ]:
#scaled dataframe - df_scd
df_scd.head(5)

In [ ]:
j = df_scd["priceUSD_h"]
k = df_scd["priceUSD_e"]
l = df_scd["priceUSD_b"]

In [ ]:
corr_h_e = j.corr(k)
corr_h_b = j.corr(l)

In [ ]:
df_scd['corr_h_e'] = (df_scd['index']*corr_h_e/-100)+0.5
df_scd['corr_h_b'] = (df_scd['index']*corr_h_b/-100)+0.5

In [ ]:
c_h_e = df_scd['corr_h_e']
c_h_b = df_scd['corr_h_b']

In [ ]:
plt.figure(figsize=(20,7))

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)
ax1.plot(time, j, color=color1)
ax1.set_xlabel('date')
ax1.grid(True)
ax1.set_title('HOPR and ETH scaled daily USD price')

ax2 = ax1
ax2.plot(time, k, color=color3)

ax3 = ax1
ax2.plot(time, c_h_e, color="black", linestyle='dashed')

fig1.set_tight_layout(True)
plt.show()

In [ ]:
#subset of data identifying the turning point of the correlation trend

In [ ]:
df_scd["wh"] = df_scd["priceUSD_h"] == df_scd["priceUSD_e"]

In [ ]:
df_scd2 = (df_scd.sort_values(by="date_r")).reset_index()

In [ ]:
alt_df = df_scd2.iloc[0:35, ]

In [ ]:
alt_df['priceUSD_h'].corr(alt_df['priceUSD_e'])

#### HOPR - BTC price 

In [ ]:
fig, axs = plt.subplots(2, sharex = True, figsize=(12, 8))
axs[0].plot(time, d_h, color=color1)
axs[0].set_title('HOPR daily USD price')
axs[1].plot(time, d_b, color=color3)
axs[1].set_title('BTC daily USD price')

In [ ]:
plt.figure(figsize=(20,7))

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)
ax1.plot(time, d_h, color=color1)
ax1.set_xlabel('date')
ax1.set_ylabel('HOPR price in USD', color=color1)
ax1.grid(True)

ax2 = ax1.twinx()
ax2.plot(time, d_b, color=color3)
ax2.set_ylabel("BTC price in USD", color=color3)

fig1.set_tight_layout(True)
plt.show()

In [ ]:
plt.figure(figsize=(20,7))

fig1 = plt.figure(1)
ax1 = fig1.add_subplot(111)
ax1.plot(time, j, color=color1)
ax1.set_xlabel('date')
ax1.grid(True)
ax1.set_title('HOPR and BTC scaled daily USD price')

ax2 = ax1
ax2.plot(time, l, color=color3)

ax3 = ax1
ax2.plot(time, c_h_b, color="black", linestyle='dashed')

fig1.set_tight_layout(True)
plt.show()

## HOPR and Twitter activity

#### Source: Most popular tweets between the launch of HOPR and middle of April.

In [ ]:
path = "C:/Users/szany/Desktop/horp/"

In [ ]:
t1 = pd.read_csv(path + '/tweet_activity_metrics_hoprnet_20210201_20210301_en.csv')
t2 = pd.read_csv(path + '/tweet_activity_metrics_hoprnet_20210301_20210401_en.csv')
t3 = pd.read_csv(path + '/tweet_activity_metrics_hoprnet_20210401_20210414_en.csv')

In [ ]:
tf_0 = t1.append([t2], ignore_index=True)
tf = tf_0.append([t3], ignore_index=True)

In [ ]:
#sorted dataframe to identify the most popular tweets 

In [ ]:
tf_s = tf.sort_values('likes', ascending=False, ignore_index=True)

In [ ]:
tf_s.head(10)

In [ ]:
a = t.index(date(2021, 2, 28))
b = t.index(date(2021, 3, 1))
c = t.index(date(2021, 3, 14))
d = t.index(date(2021, 3, 26))
e = t.index(date(2021, 3, 28))
f = t.index(date(2021, 3, 30))
h = t.index(date(2021, 4, 4))

In [ ]:
le = 1.2
ls = 0.45

fig, (ax) = plt.subplots(figsize=(20, 7))
ax.plot(t, d_h, color=color1)
ax.set_title('HOPR daily USD price')
ax.vlines(t[a], ls, le, linestyle='dashed', color='y')
ax.text(t[a], y=le, s='Get HOPR', alpha=0.7, color='y')
ax.vlines(t[b], ls, le, linestyle='dashed', color='b')
ax.text(t[b], y=(le-0.05), s='HOPR-DAI in top 20', alpha=0.7, color='b')
ax.vlines(t[c], ls, le, linestyle='dashed', color='g')
ax.text(t[c], y=(le-0.1), s='The founders', alpha=0.7, color='g')
ax.vlines(t[d], ls, le, linestyle='dashed', color='r')
ax.text(t[d], y=(le-0.15), s='CZ_binance', alpha=0.7, color='r')
ax.vlines(t[e], ls, le, linestyle='dashed', color='m')
ax.text(t[e], y=(le-0.2), s='HOPR mining', alpha=0.7, color='m')
ax.vlines(t[f], ls, le, linestyle='dashed', color='c')
ax.text(t[f], y=(le-0.25), s='HOPR Farm', alpha=0.7, color='c')
ax.vlines(t[h], ls, le, linestyle='dashed', color='k')
ax.text(t[h], y=(le-0.3), s='HOPR & DeFi', alpha=0.7, color='k')
plt.show()

In [ ]:
le = 13000000
ls = 0

fig, (ax) = plt.subplots(figsize=(20, 7))
ax.bar(t, usdvol, color=color1)
ax.set_title('HOPR daily trading volume in USD')
ax.vlines(t[a], ls, le, linestyle='dashed', color='y')
ax.text(t[a], y=le, s='Get HOPR', alpha=0.7, color='y')
ax.vlines(t[b], ls, le, linestyle='dashed', color='b')
ax.text(t[b], y=(le-1000000), s='HOPR-DAI in top 20', alpha=0.7, color='b')
ax.vlines(t[c], ls, le, linestyle='dashed', color='g')
ax.text(t[c], y=(le-2000000), s='The founders', alpha=0.7, color='g')
ax.vlines(t[d], ls, le, linestyle='dashed', color='r')
ax.text(t[d], y=(le-3000000), s='CZ_binance', alpha=0.7, color='r')
ax.vlines(t[e], ls, le, linestyle='dashed', color='m')
ax.text(t[e], y=(le-4000000), s='HOPR mining', alpha=0.7, color='m')
ax.vlines(t[f], ls, le, linestyle='dashed', color='c')
ax.text(t[f], y=(le-5000000), s='HOPR Farm', alpha=0.7, color='c')
ax.vlines(t[h], ls, le, linestyle='dashed', color='k')
ax.text(t[h], y=(le-6000000), s='HOPR & DeFi', alpha=0.7, color='k')
plt.show()

In [ ]:
le = 1300
ls = 0

fig, (ax) = plt.subplots(figsize=(20, 7))
ax.bar(t, Txns, color=color1)
ax.set_title('HOPR daily number of transactions')
ax.vlines(t[a], ls, le, linestyle='dashed', color='y')
ax.text(t[a], y=le, s='Get HOPR', alpha=0.7, color='y')
ax.vlines(t[b], ls, le, linestyle='dashed', color='b')
ax.text(t[b], y=(le-100), s='HOPR-DAI in top 20', alpha=0.7, color='b')
ax.vlines(t[c], ls, le, linestyle='dashed', color='g')
ax.text(t[c], y=(le-200), s='The founders', alpha=0.7, color='g')
ax.vlines(t[d], ls, le, linestyle='dashed', color='r')
ax.text(t[d], y=(le-300), s='CZ_binance', alpha=0.7, color='r')
ax.vlines(t[e], ls, le, linestyle='dashed', color='m')
ax.text(t[e], y=(le-400), s='HOPR mining', alpha=0.7, color='m')
ax.vlines(t[f], ls, le, linestyle='dashed', color='c')
ax.text(t[f], y=(le-500), s='HOPR Farm', alpha=0.7, color='c')
ax.vlines(t[h], ls, le, linestyle='dashed', color='k')
ax.text(t[h], y=(le-600), s='HOPR & DeFi', alpha=0.7, color='k')
plt.show()

## Traders activity

#### Source: swaps in a 2 weeks period, 1-1 week before and after the CZ_Binance post.

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

data_swb = '''
{
  swaps(
    orderBy: timestamp,
    orderDirection: asc,
    first: 1000,
    where: {
      pair: "0x92c2fc5f306405eab0ff0958f6d85d7f8892cf4d",
      timestamp_gte: 1616154120
      timestamp_lte: 1616758920
    }
  ) {
    transaction {
      id
      blockNumber
    }
    timestamp
    sender
    to
    amount0In
    amount1In
    amount0Out
    amount1Out
    amountUSD
  }
}
'''

response_swb = requests.post(url,json={'query': data_swb})
data_swb = json.loads(response_swb.text)

In [ ]:
df_swb = pd.DataFrame((pd.DataFrame(data_swb)).iloc[0][0])

In [ ]:
df_swb["cz_time"] = "before"

In [ ]:
#dataframe the week before CZ_binance - df_swb
df_swb.head(5)

In [ ]:
url = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

data_swa = '''
{
  swaps(
    orderBy: timestamp,
    orderDirection: asc,
    first: 1000,
    where: {
      pair: "0x92c2fc5f306405eab0ff0958f6d85d7f8892cf4d",
      timestamp_gte: 1616758980
      timestamp_lte: 1617360180
    }
  ) {
    transaction {
      id
      blockNumber
    }
    timestamp
    sender
    to
    amount0In
    amount1In
    amount0Out
    amount1Out
    amountUSD
  }
}
'''

response_swa = requests.post(url,json={'query': data_swa})
data_swa = json.loads(response_swa.text)

In [ ]:
df_swa = pd.DataFrame((pd.DataFrame(data_swa)).iloc[0][0])

In [ ]:
df_swa["cz_time"] = "after"

In [ ]:
#the week after the cz_binance - df_swa
df_swa.head(5)

In [ ]:
#merged dataframe about the time period of 19/03/2021-02/04/2021 - df-sw

In [ ]:
df_sw = df_swb.append([df_swa],ignore_index=True)

In [ ]:
df_sw[['amount0In', 'amount0Out', 'amount1In', 'amount1Out', 'amountUSD', 'timestamp']] = df_sw[['amount0In', 'amount0Out', 'amount1In', 'amount1Out', 'amountUSD', 'timestamp']].apply(pd.to_numeric)

In [ ]:
df_sw['s_price'] = df_sw['amount0Out']/df_sw['amount1In']
df_sw['b_price'] = df_sw['amount0In']/df_sw['amount1Out']

In [ ]:
df_sw['s_price']=df_sw['s_price'].fillna(0)
df_sw['b_price']=df_sw['b_price'].fillna(0)

In [ ]:
#added variables: price ('price'), transaction category - buy or sell ('trans_cat'), price category('amountUSD_cat'), date format time ('time_r')

In [ ]:
df_sw['price'] = df_sw['s_price']+df_sw['b_price']

In [ ]:
df_sw['trans_cat'] = df_sw['b_price'].apply(lambda x:'buy' if x > 0 else 'sell')

In [ ]:
df_sw['amountUSD_cat'] = df_sw['amountUSD'].apply(lambda x:'<$1000' if x < 1000 else ('<$2000' if x >= 1000 and x < 2000 else ('<$5000' if x >= 2000 and x < 5000 else (' <$10000' if x >= 5000 and x < 10000 else '>=$10000'))))

In [ ]:
df_sw['time_r'] = pd.to_datetime(df_sw['timestamp'], unit='s').apply(lambda x: x.date())

In [ ]:
df_sw1 = df_sw[['time_r', 'trans_cat', 'sender', "to", 'amountUSD', 'amountUSD_cat', 'price', 'transaction', 'cz_time']]

In [ ]:
#dropping the transactions where the sender and the receiver are identical

In [ ]:
df_sw1['sen_to_equal'] = df_sw1["sender"] == df_sw1["to"]

In [ ]:
double_check = df_sw1.groupby(['sen_to_equal']).count()
double_check

In [ ]:
doubles = df_sw1[df_sw1['sen_to_equal'] == True].index

In [ ]:
df_sw2 = (df_sw1.drop(doubles)).reset_index(drop=True)

In [ ]:
##dataframe about the transactions between 03/19-04/02 - df_sw2

In [ ]:
df_sw2.head(5)

# Before and after CZ_binance dataset

### Traders activity

In [ ]:
df_sw_tr_vol = df_sw2.groupby(['to']).agg({'amountUSD': ['sum']} )
df_sw_tr_act = df_sw2.groupby(['to']).agg({'to': ['count']} )

In [ ]:
df_sw_tr_volb = df_sw_tr_vol.reset_index()
df_sw_tr_actb = df_sw_tr_act.reset_index()

In [ ]:
df_sw_tr_volb.columns = ["to", "amountUSD_sum"]
df_sw_tr_actb.columns = ["to", "transaction_count"]

In [ ]:
top_volume = (df_sw_tr_volb.sort_values(by=["amountUSD_sum"], ascending=False)).reset_index(drop=True)
top_activity = (df_sw_tr_actb.sort_values(by=["transaction_count"], ascending=False)).reset_index(drop=True)

In [ ]:
#Top trades by the number of transactions ("top_activity") and the aggregated amount of trading volume ("top_volume")
top_volume.head(5)

In [ ]:
top_activity.head(5)

In [ ]:
#based on the weight of the Nr1 trader, the top 10 trader's chart includes only the [1:11] range 

In [ ]:
t_v = (top_volume.drop(columns=['amountUSD_sum'])).iloc[1:11, ]
t_a = (top_activity.drop(columns=['transaction_count'])).iloc[1:11, ]

In [ ]:
#the overlap between the first 10 most active and biggest volume traders 

In [ ]:
overlap = (t_v.to.isin(t_a.to).astype(int)).sum()
overlap

In [ ]:
t_v["top_traders_v"] = "top_vol"
t_a["top_traders_a"] = "top_act"

In [ ]:
t_v.columns=["to", "top_traders_v"]
t_a.columns=["to", "top_traders_a"]

In [ ]:
df_v = pd.merge(df_sw2, t_v, how="left", on=["to"])

In [ ]:
df_va = pd.merge(df_v, t_a, how="left", on=["to"])

In [ ]:
t_v_p = df_va[(df_va["top_traders_v"] == "top_vol")]
t_a_p = df_va[(df_va["top_traders_a"] == "top_act")]

In [ ]:
def xyz(col):
    if col['trans_cat'] == "sell":
        val = col['amountUSD']*-1
    else:
        val = col['amountUSD']
    return val

In [ ]:
t_v_p['amountUSD_pos'] = t_v_p.apply(xyz, axis=1)

In [ ]:
#subset of transactions, initiated by the biggest traders - t_v_p, t_v_a
t_v_p.head(5)

In [ ]:
#the waterfall chart includes the trading activity of the biggest (by volume) HOPR traders by volume

In [ ]:
wat_fal_v = t_v_p.groupby(["time_r", "trans_cat"]).agg({'amountUSD_pos': ['sum']} )

In [ ]:
wat_fal_v2 = wat_fal_v.reset_index()
wat_fal_v2.columns = ["time_r", "trans_cat", "amountUSD_pos"]

In [ ]:
from waterfall_ax import WaterfallChart

In [ ]:
wat_fal_v2["amountUSD_pos2"] = wat_fal_v2["amountUSD_pos"]/1000

In [ ]:
ab = list(wat_fal_v2.amountUSD_pos2.cumsum())

In [ ]:
plt.style.use('ggplot')

step_names = [' 19','19 ', '20 ', '20 ', ' 21', '21 ', ' 22', '22 ', ' 23', '23 ', ' 24', '24 ', ' 25', '25 ', ' 26', '26 ', ' 27', '27 ', ' 28', '28 ', ' 29', '29 ', ' 30', '30 ', ' 31', '31 ', ' 1', '1 ', ' 2', ' ']
metric_name = 'amount in USD (1000)'
bar_kwargs = {'edgecolor': color1}
line_kwargs = {'color': color3}

# Plot
waterfall = WaterfallChart(ab, step_names= step_names, metric_name= metric_name)

#waterfall = WaterfallChart(ab, step_names = step_names, metric_name = metric_name,last_step_label = last_step_label)
wf_ax = waterfall.plot_waterfall(title='Biggest volume traders', bar_labels = False, bar_kwargs = bar_kwargs, line_kwargs = line_kwargs)

### Daily trading activity

In [ ]:
after_czb00 = df_sw[(df_sw["cz_time"] == "after")]
before_czb00 = df_sw[(df_sw["cz_time"] == "before")]

In [ ]:
after_czb0 = (after_czb00.sort_values(by=['time_r'])).reset_index()
before_czb0 = (before_czb00.sort_values(by=['time_r'])).reset_index()

In [ ]:
after_czb = after_czb0[['time_r', 'trans_cat', 'sender', "to", 'amountUSD', 'amountUSD_cat', 'price', 'transaction', 'index']]
before_czb = before_czb0[['time_r', 'trans_cat', 'sender', "to", 'amountUSD', 'amountUSD_cat', 'price', 'transaction', 'index']]

In [ ]:
#subset of datasets separated by CZ_Binance tweet - after_czb, before_czb
after_czb.head(5)

In [ ]:
after_usd_s = after_czb.groupby(['time_r', 'amountUSD_cat']).agg({'amountUSD': ['sum']} ).unstack('amountUSD_cat')
before_usd_s = before_czb.groupby(['time_r', 'amountUSD_cat']).agg({'amountUSD': ['sum']} ).unstack('amountUSD_cat')

In [ ]:
after_usd_s.columns = ['<$10000', '<$1000', '<$2000', '<$5000','>=$10000']
before_usd_s.columns = ['<$10000', '<$1000', '<$2000', '<$5000','>=$10000']

In [ ]:
# composition of the trading activity before and after CZ_Binance, by the volume of the trading
#(consider the non-linear structure of the labeling!)

In [ ]:
before_usd_s.plot.bar(stacked=True, figsize=(10,5))
after_usd_s.plot.bar(stacked=True, figsize=(10,5))

In [ ]:
# composition of the trading activity before and after CZ_Binance, by the normalized volume of the trading
#(consider the non-linear structure of the labeling!)

In [ ]:
after_usd_s2 = after_usd_s.div(after_usd_s.sum(1), axis=0)
before_usd_s2 = before_usd_s.div(before_usd_s.sum(1), axis=0)

In [ ]:
before_usd_s2.plot.bar(stacked=True, figsize=(10,5))
after_usd_s2.plot.bar(stacked=True, figsize=(10,5))

In [ ]:
# composition of the trading activity before and after CZ_Binance, by the normalized number of transactions
#(consider the non-linear structure of the labeling!)

In [ ]:
after_usd_c = after_czb.groupby('time_r')['amountUSD_cat'].value_counts(normalize=True).unstack('amountUSD_cat')
before_usd_c = before_czb.groupby('time_r')['amountUSD_cat'].value_counts(normalize=True).unstack('amountUSD_cat')

In [ ]:
before_usd_c.plot.bar(stacked=True, figsize=(10,5))
after_usd_c.plot.bar(stacked=True, figsize=(10,5))

### Conclusions

HOPR - ETH - BTC

HOPR has a strong, negative correlation with ETH.

However, based on the scaled dataset, the trend in the correlation between HOPR and ETH was weak positive until the begining of April, when it flipped. 

HOPR has moderate, positive correlation with BTW. 

CZ_Binance effect

The CZ_Binance tweet influenced the price, the trading volume, and the number of transactions.

The immediate effect was the strongest on the 29th and 30th of March, when besides the increase in the price, the volume of the trade doubled (2 times higher on the 29th and 1,5 times higher on the 30th) compared to the days before the tweet. 

The tweet also influenced the structure of the trading. The increase of the trading volume was caused mostly by “big players” (10000$+ ) but it raised the attention of the 5000$ -10000$ group as well, which is more visible in the number of transactions. 

5 traders out of the top 10 highest volume trader also appears in the top 10 most active trader (measured by the number of transaction initiated within the given period)  
